In [279]:
import re
import string
import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from sklearn.linear_model import LogisticRegression
from nltk.stem import SnowballStemmer

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
stemmer = SnowballStemmer('russian') 
sw = stopwords.words('russian')   

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim: int, hidden_size:int, embedding: torch.nn.modules.sparse.Embedding) -> None:
        super().__init__()

        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            batch_first=True
        )
        self.clf = nn.Linear(self.hidden_size, 6)  # Изменено для предсказания 6 классов

    def forward(self, x):
        embeddings = self.embedding(x)
        _, (h_n, _) = self.lstm(embeddings)
        out = self.clf(h_n.squeeze())
        return out


def data_preprocessing(text: str) -> str:
    """preprocessing string: lowercase, removing html-tags, punctuation, 
                            stopwords, digits

    Args:
        text (str): input string for preprocessing

    Returns:
        str: preprocessed string
    """    

    text = text.lower()
    text = re.sub('<.*?>', '', text) # html tags
    text = ''.join([c for c in text if c not in string.punctuation])# Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = [word for word in text.split() if not word.isdigit()]
    text = ' '.join(text)
    return text

def get_words_by_freq(sorted_words: list, n: int = 10) -> list:
    return list(filter(lambda x: x[1] > n, sorted_words))

def padding(review_int: list, seq_len: int) -> np.array: # type: ignore
    """Make left-sided padding for input list of tokens

    Args:
        review_int (list): input list of tokens
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros

    Returns:
        np.array: padded sequences
    """    
    features = np.zeros((len(review_int), seq_len), dtype = int)
    for i, review in enumerate(review_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            new = zeros + review
        else:
            new = review[: seq_len]
        features[i, :] = np.array(new)
            
    return features

def preprocess_single_string(
    input_string: str, 
    seq_len: int, 
    vocab_to_int: dict,
    ) -> torch.tensor:
    """Function for all preprocessing steps on a single string

    Args:
        input_string (str): input single string for preprocessing
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros
        vocab_to_int (dict, optional): word corpus {'word' : int index}. Defaults to vocab_to_int.

    Returns:
        list: preprocessed string
    """    

    preprocessed_string = data_preprocessing(input_string)
    result_list = []
    for word in preprocessed_string.split():
        try: 
            result_list.append(vocab_to_int[word])
        except KeyError as e:
            print(f'{e}: not in dictionary!')
    result_padded = padding([result_list], seq_len)[0]

    return torch.tensor(result_padded)

def predict_sentence(text: str, model: nn.Module, seq_len: int, vocab_to_int: dict) -> str:
    rating_dict = {
        0: "Отвратительно! Даже не подходите к этому месту!",
        1: "Плохо! Лучше бы остался дома.",
        2: "Удовлетворительно, но не без недостатков. Ешьте на свой страх и риск.",
        3: "Хорошо! Вполне достойное место для трапезы.",
        4: "Отлично! Обязательно вернусь еще раз.",
        5: "Великолепно! Как в раю, только с едой."
    }
    
    p_str = preprocess_single_string(text, seq_len, vocab_to_int).unsqueeze(0)
    model.eval()
    pred = model(p_str)
    output = pred.argmax(dim=1).item()
    return rating_dict[output]

def predict_single_string(text: str,
                          model: BertModel,
                          loaded_model: LogisticRegression
) -> str:
    rating_dict = {
        0: "Отвратительно! Даже не подходите к этому месту!",
        1: "Плохо! Лучше бы остался дома.",
        2: "Удовлетворительно, но не без недостатков. Ешьте на свой страх и риск.",
        3: "Хорошо! Вполне достойное место для трапезы.",
        4: "Отлично! Обязательно вернусь еще раз.",
        5: "Великолепно! Как в раю, только с едой."
    }

    with torch.no_grad():
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        vector = output[0][:,0,:]
        prediction = loaded_model.predict(vector)[0]
    return rating_dict[prediction]

def clean(text):

    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # заменить два и более пробела на один пробел
    text = re.sub(r'\d+', ' ', text) # удаляем числа
    text = text.translate(str.maketrans('', '', string.punctuation)) # удаляем знаки пунктуации 
    text = re.sub(r'\n+', ' ', text) # удаляем символ перевод строки 
    
    return text

def tokin(text):
    text = clean(text)
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    text = ' '.join([word for word in text.split() if word not in sw])
    return text

def predict_ml_class(text, loaded_vectorizer, loaded_classifier):
    rating_dict = {
        0: "Отвратительно! Даже не подходите к этому месту!",
        1: "Плохо! Лучше бы остался дома.",
        2: "Удовлетворительно, но не без недостатков. Ешьте на свой страх и риск.",
        3: "Хорошо! Вполне достойное место для трапезы.",
        4: "Отлично! Обязательно вернусь еще раз.",
        5: "Великолепно! Как в раю, только с едой."
    }
    
    t = tokin(text).split(' ')
    new_text_bow = loaded_vectorizer.transform(t)
    predicted_label = loaded_classifier.predict(new_text_bow)[0]
    return rating_dict[predicted_label]


/Users/valeriaalesnikova/Desktop/bootcamp/nlp_project-1/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/valeriaalesnikova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [254]:
import os
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
import json
from train_rnn import train_rnn_multiclass
from sklearn.model_selection import train_test_split
import torchutils as tu
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torchmetrics.classification import Accuracy

In [255]:
# # Лучше всего установить такую же версию
# print(gensim.__version__)

In [256]:
# Загрузка данных
file_path = '/Users/valeriaalesnikova/Desktop/bootcamp/nlp_project-1/models/restaurants_reviews.jsonl'
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [257]:
def data_preprocessing(text: str) -> str:
    """preprocessing string: lowercase, removing html-tags, punctuation and stopwords

    Args:
        text (str): input string for preprocessing

    Returns:
        str: preprocessed string
    """    

    text = text.lower()
    text = re.sub('<.*?>', '', text) # html tags
    text = ''.join([c for c in text if c not in string.punctuation])# Remove punctuation
    text = [word for word in text.split() if word not in stop_words] 
    text = ' '.join(text)
    return text

df['cleaned_text'] = df['text']

In [258]:
df.head(15)

,review_id,general,food,interior,service,text,cleaned_text
0,0,0,10,10,10,Вытянули меня сегодня в город и раз уж была в ...,Вытянули меня сегодня в город и раз уж была в ...
1,1,0,9,10,9,проводили корпоратив на 60 чел. в этот - уже т...,проводили корпоратив на 60 чел. в этот - уже т...
2,2,0,9,10,10,Был в Гостях с женой один раз и еще раз с жено...,Был в Гостях с женой один раз и еще раз с жено...
3,3,0,-,5,10,Бар понравился на первый взгляд . Интерьер к ...,Бар понравился на первый взгляд . Интерьер к ...
4,4,0,7,10,10,В « Bel Canto » мы отмечали юбилей моего отца ...,В « Bel Canto » мы отмечали юбилей моего отца ...
5,5,0,10,10,10,"Здравствуйте , уважаемые женихи и невесты . Г...","Здравствуйте , уважаемые женихи и невесты . Г..."
6,6,0,10,10,10,"Вкусная кухня , уютная остановка , хорошее обс...","Вкусная кухня , уютная остановка , хорошее обс..."
7,7,0,10,10,10,"Вкусная кухня , интересная обстановка , отличн...","Вкусная кухня , интересная обстановка , отличн..."
8,8,0,1,-,1,8 июня заказали 2 пиццы - Бергамо и Юнет . Пи...,8 июня заказали 2 пиццы - Бергамо и Юнет . Пи...
9,9,0,-,5,7,Началась жаркая пора в нашем летнем расписании...,Началась жаркая пора в нашем летнем расписании...


In [259]:
corpus = [word for text in df['cleaned_text'] for word in text.split()]
count_words = Counter(corpus)

sorted_words = count_words.most_common()


In [260]:
def get_words_by_freq(sorted_words: list, n: int = 10) -> list:
    return list(filter(lambda x: x[1] > n, sorted_words))

In [261]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

In [262]:
reviews_int = []
for text in df['cleaned_text']:

    r = [vocab_to_int[word] for word in text.split() if vocab_to_int.get(word)]
    reviews_int.append(r)

In [263]:
# df['general'] = df['general'].apply(lambda x: 1 if x == 'positive' else 0)

In [264]:
review_len = [len(x) for x in reviews_int]
df['Review len'] = review_len
df.head()

,review_id,general,food,interior,service,text,cleaned_text,Review len
0,0,0,10,10,10,Вытянули меня сегодня в город и раз уж была в ...,Вытянули меня сегодня в город и раз уж была в ...,330
1,1,0,9,10,9,проводили корпоратив на 60 чел. в этот - уже т...,проводили корпоратив на 60 чел. в этот - уже т...,158
2,2,0,9,10,10,Был в Гостях с женой один раз и еще раз с жено...,Был в Гостях с женой один раз и еще раз с жено...,103
3,3,0,-,5,10,Бар понравился на первый взгляд . Интерьер к ...,Бар понравился на первый взгляд . Интерьер к ...,157
4,4,0,7,10,10,В « Bel Canto » мы отмечали юбилей моего отца ...,В « Bel Canto » мы отмечали юбилей моего отца ...,354


In [265]:
def padding(review_int: list, seq_len: int) -> np.array:
    """Make left-sided padding for input list of tokens

    Args:
        review_int (list): input list of tokens
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros

    Returns:
        np.array: padded sequences
    """    
    features = np.zeros((len(reviews_int), seq_len), dtype = int)
    for i, review in enumerate(review_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            new = zeros + review
        else:
            new = review[: seq_len]
        features[i, :] = np.array(new)
            
    return features

In [266]:
def preprocess_single_string(input_string: str, seq_len: int, vocab_to_int: dict = vocab_to_int) -> list:
    """Function for all preprocessing steps on a single string

    Args:
        input_string (str): input single string for preprocessing
        seq_len (int): max length of sequence, it len(review_int[i]) > seq_len it will be trimmed, else it will be padded by zeros
        vocab_to_int (dict, optional): word corpus {'word' : int index}. Defaults to vocab_to_int.

    Returns:
        list: preprocessed string
    """    

    preprocessed_string = data_preprocessing(input_string)
    result_list = []
    for word in preprocessed_string.split():
        try: 
            result_list.append(vocab_to_int[word])
        except KeyError as e:
            print(f'{e}: not in dictionary!')
    result_padded = padding([result_list], seq_len)[0]

    return torch.tensor(result_padded)

In [267]:
SEQ_LEN = 32
features = padding(reviews_int, SEQ_LEN)
print(features[3, :])

[ 3608   199     6   250   332     2   129    30 46982  1147    18     5
 17205     1    14   933  2632     1    89  3881     1  1587     2   468
  3073    28   334   950  1100     8  6393     1]


In [268]:
X_train, X_valid, y_train, y_valid = train_test_split(features, df['general'].to_numpy(), test_size=0.2, random_state=1)

In [269]:

train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid))


BATCH_SIZE = 32

train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)

In [270]:
VOCAB_SIZE = len(vocab_to_int)+1 

In [271]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)

In [272]:
device='cpu'

In [273]:
@dataclass
class ConfigRNN:
    vocab_size: int
    device: str
    n_layers: int
    embedding_dim: int
    hidden_size: int
    seq_len: int
    bidirectional: bool or int

In [274]:
# Конфигурация модели (пример)
class ConfigRNN:
    def __init__(self, vocab_size, device, n_layers, embedding_dim, hidden_size, seq_len, bidirectional):
        self.vocab_size = vocab_size
        self.device = device
        self.n_layers = n_layers
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.bidirectional = bidirectional

# Пример словаря и длины последовательности
vocab_to_int = {
    0: "Отвратительно! Даже не подходите к этому месту!",
    1: "Плохо! Лучше бы остался дома.",
    2: "Удовлетворительно, но не без недостатков. Ешьте на свой страх и риск.",
    3: "Хорошо! Вполне достойное место для трапезы.",
    4: "Отлично! Обязательно вернусь еще раз.",
    5: "Великолепно! Как в раю, только с едой.",
}  # пример словаря
SEQ_LEN = 10

In [275]:
net_config = ConfigRNN(
    vocab_size=len(vocab_to_int) + 1,
    device="cpu",
    n_layers=2,
    embedding_dim=16,
    hidden_size=32,
    seq_len=SEQ_LEN,
    bidirectional=False,
)
net_config

In [278]:
class RNNNet(nn.Module):
    """
    vocab_size: int, размер словаря (аргумент embedding-слоя)
    emb_size:   int, размер вектора для описания каждого элемента последовательности
    hidden_dim: int, размер вектора скрытого состояния, default 0
    batch_size: int, размер batch
    """

    class RNNNet(nn.Module):
        def __init__(self, rnn_conf) -> None:
            super().__init__()
            self.rnn_conf = rnn_conf
            self.seq_len = rnn_conf.seq_len
            self.emb_size = rnn_conf.embedding_dim
            self.hidden_dim = rnn_conf.hidden_size
            self.n_layers = rnn_conf.n_layers
            self.vocab_size = rnn_conf.vocab_size
            self.bidirectional = rnn_conf.bidirectional

            self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
            self.rnn_cell = nn.RNN(
                input_size=self.emb_size,
                hidden_size=self.hidden_dim,
                batch_first=True,
                bidirectional=self.bidirectional,
                num_layers=self.n_layers,
            )

            self.bidirect_factor = 2 if self.bidirectional else 1
            self.linear = nn.Sequential(
                nn.Linear(self.hidden_dim * self.seq_len * self.bidirect_factor, 16),
                nn.Tanh(),
                nn.Linear(16, 5)
            )

    def model_description(self):
        direction = "bidirect" if self.bidirectional else "onedirect"
        return f"rnn_{direction}_{self.n_layers}"

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x.to(self.rnn_conf.device))
        output, _ = self.rnn_cell(x)  # Забираем hidden states со всех промежуточных состояний, второй выход отправляем в _
        output = output.contiguous().view(output.size(0), -1)
        out = self.linear(output)
        return out


# Создание экземпляра модели
model_rnn = RNNNet(net_config)
tu.get_model_summary(model_rnn, sample_x.to(net_config.device))

TypeError: RNNNet.__init__() takes 1 positional argument but 2 were given

In [ ]:
import torch.optim as optim
from torch.optim import Adam

In [ ]:
# Инициализация функции потерь, оптимизатора и метрики
criterion = nn.CrossEntropyLoss()
optimizer_rnn = Adam(model_rnn.parameters(), lr=0.001)
metric = Accuracy(task='multiclass', num_classes=5).to(net_config.device)

In [ ]:
# цикл обучения и валидации 4,30 минуты

train_losses_rnn, val_losses_rnn, train_metric_rnn, val_metric_rnn, rnn_time = train_rnn_multiclass(
    epochs=5,
    model=model_rnn,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer_rnn,
    rnn_conf=net_config,
    criterion=criterion,
    metric=metric,
)